In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler

# 创建一个4比特的量子电路和1比特的经典寄存器，用于记录辅助比特的测量结果
qc = QuantumCircuit(4, 4)

# 假设比特1,2,3已经被你制备成需要的叠加态
# 例如，你可以在此处添加状态制备代码
# qc.ry(..., 1)
# qc.ry(..., 2)
# qc.ry(..., 3)

# 1. 对辅助比特（0号）施加 Hadamard 门
qc.h(0)

qc.x(0)
qc.h(1)
# 2. 以辅助比特为控制位，对每个测试比特施加 CZ 门
for target in [1, 2, 3]:
    qc.cz(0, target)

# 3. 再对辅助比特施加 Hadamard 门
qc.h(0)

import cmath
import numpy as np
statevector = Statevector(qc)
# 获取基态标签（例如 '0000', '0001', ...）
n = int(np.log2(len(statevector)))
basis = [format(i, f'0{n}b') for i in range(len(statevector))]
# 打印每个基态的幅度和相位
for i, amplitude in enumerate(statevector):
    r = abs(amplitude)
    theta = cmath.phase(amplitude)
    if r > 1e-10:  # 过滤掉非常接近0的项
        print(f"|{basis[i]}>: {r:.4f} * e^(i {theta:.4f})")





# 4. 测量辅助比特到经典寄存器（仅测量辅助比特）
qc.measure(range(4), range(4))

# 打印电路
print(qc.draw())

sampler = Sampler(mode=AerSimulator())
job = sampler.run([qc], shots=1024)

# 获取结果
result = job.result()
pub_result = result[0]

# 提取每次测量的结果
counts = pub_result.data.c.get_counts()
print("Measurement counts:", counts)


Statevector: Statevector([7.07106781e-01+0.j, 2.29934717e-17+0.j, 2.29934717e-17+0.j,
             7.07106781e-01+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j,
             0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j,
             0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j,
             0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j,
             0.00000000e+00+0.j],
            dims=(2, 2, 2, 2))
|0000>: 0.7071 * e^(i 0.0000)
|0011>: 0.7071 * e^(i 0.0000)
     ┌───┐┌───┐            ┌───┐   ┌─┐
q_0: ┤ H ├┤ X ├─■──■─────■─┤ H ├───┤M├
     ├───┤└───┘ │  │ ┌─┐ │ └───┘   └╥┘
q_1: ┤ H ├──────■──┼─┤M├─┼──────────╫─
     └───┘         │ └╥┘ │  ┌─┐     ║ 
q_2: ──────────────■──╫──┼──┤M├─────╫─
                      ║  │  └╥┘ ┌─┐ ║ 
q_3: ─────────────────╫──■───╫──┤M├─╫─
                      ║      ║  └╥┘ ║ 
c: 4/═════════════════╩══════╩═══╩══╩═
                      1      2   3  0 
Measurement counts: {'0000': 527, '0011': 497}


In [3]:
import numpy as np

A = np.array([[1, 2, -3], [1, 0, 1]])  # 系统矩阵
b = np.array([0, 2])           # 常数向量

# 使用最小二乘法求解 Ax = b
x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
print("特解为：", x)


特解为： [1.33333333 0.33333333 0.66666667]


In [4]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler

# 创建量子电路
qc = QuantumCircuit(5, 5)
for i in range(1000):
    qc.h(0)
    qc.measure(range(1), range(1))
    qc.h(0) 
    qc.measure(range(1), range(1))
    qc.h(1)
    qc.h(1) 
    qc.h(2)
    qc.h(2) 

# 打印电路
print(qc.draw())

# 初始化模拟器
aer = AerSimulator()
CORE_BASIS_GATES = ["measure", "cx", "id", "rz", "sx", "x"]
generate_preset_pass_manager(optimization_level=2, basis_gates=CORE_BASIS_GATES)

# 创建采样器并运行电路
sampler = Sampler(mode=aer)
job = sampler.run([qc], shots=1024)

# 获取结果
result = job.result()
pub_result = result[0]

# 提取每次测量的结果
counts = pub_result.data.c.get_counts()
print("Measurement counts:", counts)


     ┌───┐┌─┐┌───┐     ┌─┐┌───┐     ┌─┐┌───┐     ┌─┐┌───┐     ┌─┐┌───┐     ┌─┐»
q_0: ┤ H ├┤M├┤ H ├─────┤M├┤ H ├─────┤M├┤ H ├─────┤M├┤ H ├─────┤M├┤ H ├─────┤M├»
     ├───┤└╥┘├───┤┌───┐└╥┘├───┤┌───┐└╥┘├───┤┌───┐└╥┘├───┤┌───┐└╥┘├───┤┌───┐└╥┘»
q_1: ┤ H ├─╫─┤ H ├┤ H ├─╫─┤ H ├┤ H ├─╫─┤ H ├┤ H ├─╫─┤ H ├┤ H ├─╫─┤ H ├┤ H ├─╫─»
     ├───┤ ║ ├───┤├───┤ ║ ├───┤├───┤ ║ ├───┤├───┤ ║ ├───┤├───┤ ║ ├───┤├───┤ ║ »
q_2: ┤ H ├─╫─┤ H ├┤ H ├─╫─┤ H ├┤ H ├─╫─┤ H ├┤ H ├─╫─┤ H ├┤ H ├─╫─┤ H ├┤ H ├─╫─»
     └───┘ ║ └───┘└───┘ ║ └───┘└───┘ ║ └───┘└───┘ ║ └───┘└───┘ ║ └───┘└───┘ ║ »
q_3: ──────╫────────────╫────────────╫────────────╫────────────╫────────────╫─»
           ║            ║            ║            ║            ║            ║ »
q_4: ──────╫────────────╫────────────╫────────────╫────────────╫────────────╫─»
           ║            ║            ║            ║            ║            ║ »
c: 5/══════╩════════════╩════════════╩════════════╩════════════╩════════════╩═»
           0            0            0  

In [5]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# 创建量子电路
num_qubits = 2
qc = QuantumCircuit(num_qubits)

# 添加量子门（示例）
qc.h(0)
qc.cx(0, 1)
# 获取理想状态向量
statevector = Statevector.from_instruction(qc)

# 计算理论概率分布
probabilities = statevector.probabilities()
import numpy as np
print(np.count_nonzero(probabilities))
# 打印结果
print(probabilities)


2
[0.5 0.  0.  0.5]


In [6]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
qc = QuantumCircuit(2, 2)
qc.x(1)
qc.measure(range(2),range(2))
qc.h(0)
qc.measure(range(2),range(2))
qc.x(0)
qc.measure(range(2),range(2))
# qc.measure_all()
print(qc.draw())
aer = AerSimulator()
CORE_BASIS_GATES = ["measure", "cx", "id", "rz", "sx", "x"]
generate_preset_pass_manager(optimization_level=2, basis_gates=CORE_BASIS_GATES,)

sampler = Sampler(mode=aer)
job = sampler.run([qc], shots=1024)
result = job.result()
pub_result = result[0]
counts = pub_result.data.c.get_counts()
print(counts)

          ┌─┐┌───┐┌─┐┌───┐   ┌─┐
q_0: ─────┤M├┤ H ├┤M├┤ X ├───┤M├
     ┌───┐└╥┘└┬─┬┘└╥┘└┬─┬┘┌─┐└╥┘
q_1: ┤ X ├─╫──┤M├──╫──┤M├─┤M├─╫─
     └───┘ ║  └╥┘  ║  └╥┘ └╥┘ ║ 
c: 2/══════╩═══╩═══╩═══╩═══╩══╩═
           0   1   0   1   1  0 
{'11': 500, '10': 524}
